## Get the data

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
from sklearn import datasets
boston_X, boston_y = datasets.load_boston(return_X_y=True)

from sklearn.linear_model import LinearRegression

## Cross validation: k-fold CV

The purpose of testing is to estimate a models quality of predicting data out of sample. For the purpose of testing a single split of the data has greater risk of of not being representative for a model's ability to generalize. Hence, where possible, multiple splits are preferred. An unordered dataset is typically split into folds. <u>Out of k folds each one is used as test set in turn</u>. There are as many train-test splits and scores as there are folds.

This maximizes the amount of data that is used to train the model, as during the course of training, the model is not only trained, but also tested on all of the available data. However, <u>the number of folds also determines the computational cost</u>.

In [12]:
from sklearn.model_selection import cross_val_score

reg = LinearRegression()
cv_results = cross_val_score(reg, boston_X, boston_y, cv=5)  # gives array of R2s
print(cv_results)

[ 0.63919994  0.71386698  0.58702344  0.07923081 -0.25294154]


## Tryouts

https://scikit-learn.org/stable/tutorial/index.html#tutorial-menu

now:
https://scikit-learn.org/stable/tutorial/statistical_inference/supervised_learning.html